In [16]:
import pandas as pd
import os

In [17]:
data_folder = 'data'
clean_data_folder = 'clean_data'

In [18]:
routes = ['convention', 'washington', 'sts']

In [19]:
filter = dict()
filter['convention'] = dict()
filter['convention']['arc_id'] = 5671 

filter['sts'] = dict()
filter['sts']['arc_id'] = 191

filter['washington'] = dict()
filter['washington']['arc_id'] = 4264

In [20]:
print(routes)

['convention', 'washington', 'sts']


In [21]:
for route in routes:        
    data = pd.read_csv(os.path.join(data_folder, os.path.join(route, route + '.csv')), delimiter=';').drop(columns=["Libelle", "Etat trafic", "Identifiant noeud amont", "Libelle noeud amont", "Identifiant noeud aval", "Libelle noeud aval", "Etat arc", "Date debut dispo data", "Date fin dispo data", "geo_point_2d", "geo_shape"])
    data = data[data['Identifiant arc'] == filter[route]['arc_id']]

    data['Date et heure de comptage'] = data['Date et heure de comptage'].astype(str).apply(lambda x : x[:-6])
    data.to_csv(os.path.join(clean_data_folder, os.path.join(route, route + '.csv')), index=False)

opendata_folder = 'opendata'
clean_opendata_folder = 'clean_opendata'
years = ['2016', '2017', '2018', '2019']

for route in routes:
    data = pd.read_csv(os.path.join(clean_data_folder, os.path.join(route, route + '.csv')))
    data['datetime'] = pd.to_datetime(data['Date et heure de comptage'])
    data = data[data['datetime'].dt.year == 2020]
    data = data.drop(columns=['datetime'])
    
    data.to_csv(os.path.join(clean_opendata_folder, os.path.join("2020", route + '.csv')), index=False)

for year in years:
    print(" [.] Working on year", year)
    routes_filtered = dict()
    for route in routes:
        routes_filtered[route] = pd.DataFrame()
    files = os.listdir(os.path.join(opendata_folder, year))
    for file in sorted(files):
        print(" [.][.] Working on file", file)
        week_data = pd.read_csv(os.path.join(opendata_folder, os.path.join(year, file)), delimiter=';').drop(columns=["dessin", "libelle", "iu_nd_amont", "libelle_nd_amont", "iu_nd_aval", "libelle_nd_aval", "etat_trafic", "etat_barre"])
        week_data = week_data.rename(columns={"iu_ac": "Identifiant arc", "q" : "Débit horaire", "k": "Taux d'occupation", "t_1h" : "Date et heure de comptage"})
        for route in routes:
            filtered_data = week_data[week_data['Identifiant arc'] == filter[route]['arc_id']]
            routes_filtered[route] = routes_filtered[route].append(filtered_data)
            
    for route in routes:
        routes_filtered[route].to_csv(os.path.join(clean_opendata_folder, os.path.join(year, route + ".csv")), index=False)